# -*- coding: utf-8 -*-
"""
Created on Tue Feb 26 16:00:07 2021

@author: 
"""
# Load Libraries

In [2]:
from pandas import read_csv
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


Load Dataset

In [3]:
dataframe = read_csv("sonar.all-data.csv", header=None)
dataset = dataframe.values
# split into input and output variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

encoding  required using the entire dataset via the fit() function,<br>
then apply the encoding to create a new output variable using the transform() function.<br>
encode class values as integers

In [4]:
one_hot_encode = pd.get_dummies(Y)
print("dummy y ", one_hot_encode)

dummy y       M  R
0    0  1
1    0  1
2    0  1
3    0  1
4    0  1
..  .. ..
203  1  0
204  1  0
205  1  0
206  1  0
207  1  0

[208 rows x 2 columns]


baseline model

In [5]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

evaluate model with dataset using stratified cross-validation in the scikit-learn<br>
framework. pass the number of training epochs to the KerasClassifier, again using<br>
reasonable default values. Verbose output is also turned 0 given that the model will be created<br>
10 times for the 10-fold cross-validation being performed.

In [6]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 82.70% (6.43%)


Improve Performance With Data standardized data using using a Pipeline class. The pipeline is a wrapper that<br>
executes one or more models within a pass of the cross-validation procedure.<br>
define a pipeline with the StandardScaler followed by our neural network model.<br>
evaluate baseline model with standardized dataset

In [7]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100,
batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 88.38% (4.08%)


Tuning Layers and Neurons in The Model using the baseline model with<br>
Here baseline model with 60 neurons in the hidden layer and reduce it by half to 30.

smaller model

In [8]:
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100,batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.02% (10.08%)


With further tuning in the model like the optimization algorithm and the number of training<br>
epochs, expected that further improvements are possible.